### **教師あり学習練習

### 授業で使用したデータを利用し、ランダムフォレストにで分類を実施しました。

In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import random
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble  import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
train_file = '/content/drive/MyDrive/Colab Notebooks/model_2016_ST4000DM000.csv'
test_file = '/content/drive/MyDrive/Colab Notebooks/model_2017_ST4000DM000.csv'

In [4]:
df = pd.read_csv(train_file)

In [5]:
#データの概要を確認
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076071 entries, 0 to 1076070
Data columns (total 95 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   date                  1076071 non-null  object 
 1   serial_number         1076071 non-null  object 
 2   model                 1076071 non-null  object 
 3   capacity_bytes        1076071 non-null  int64  
 4   failure               1076071 non-null  int64  
 5   smart_1_normalized    1076069 non-null  float64
 6   smart_1_raw           1076069 non-null  float64
 7   smart_2_normalized    0 non-null        float64
 8   smart_2_raw           0 non-null        float64
 9   smart_3_normalized    1076069 non-null  float64
 10  smart_3_raw           1076069 non-null  float64
 11  smart_4_normalized    1076069 non-null  float64
 12  smart_4_raw           1076069 non-null  float64
 13  smart_5_normalized    1076069 non-null  float64
 14  smart_5_raw           1076069 non-

In [6]:
df_t = pd.read_csv(test_file)
print(df.shape, df_t.shape)

(1076071, 95) (178298, 95)


In [7]:
df['date'] = pd.to_datetime(df['date'])
df_t['date'] = pd.to_datetime(df['date'])

In [8]:
# 欠損値データの削除（すべて欠損値のカラム，一つでも欠損値がある行）
df = df.dropna(axis = 1, how="all")
df = df.dropna(axis=0,how="any")
df.shape

(1076069, 53)

In [9]:
def remove_constant_data(df,check_col):
    col_list = df.columns.to_list()
    remain_list = []
    
    for c in col_list:
        if check_col in c:
            c_min = df[c].min()
            c_max = df[c].max()
            
            if c_min != c_max:
                print(f'{c:20s} {int(c_min):15d} {int(c_max):15d}')
                remain_list.append(c)

    return remain_list

In [10]:
col_list = remove_constant_data(df,'_raw')
col_info = ['date','serial_number','failure']
col_use = col_info + col_list

smart_1_raw                        0       244139640
smart_4_raw                        1            1216
smart_5_raw                        0           28704
smart_7_raw                       16 281471465354171
smart_9_raw                        9           29989
smart_12_raw                       0           16416
smart_183_raw                      0           21060
smart_184_raw                      0               6
smart_187_raw                      0             844
smart_188_raw                      0     60130459662
smart_189_raw                      0            6562
smart_190_raw                     13              43
smart_192_raw                      0            1208
smart_193_raw                      5          465712
smart_194_raw                     13              43
smart_197_raw                      0           13864
smart_198_raw                      0           13864
smart_199_raw                      0            5415
smart_240_raw                      4 281462091

In [11]:
df = df[col_use]
df_t = df_t[col_use]

In [12]:
# テストデータの欠損値チェック & 欠損データ（行）の削除
print(df_t.isna().sum())
print('削除前: ', df_t.shape[0])
df_t = df_t.dropna(axis=0, how='any')
print('削除後; ', df_t.shape[0])

date              0
serial_number     0
failure           0
smart_1_raw      13
smart_4_raw      13
smart_5_raw      13
smart_7_raw      13
smart_9_raw      13
smart_12_raw     13
smart_183_raw    13
smart_184_raw    13
smart_187_raw    13
smart_188_raw    13
smart_189_raw    13
smart_190_raw    13
smart_192_raw    13
smart_193_raw    13
smart_194_raw    13
smart_197_raw    13
smart_198_raw    13
smart_199_raw    13
smart_240_raw    13
smart_241_raw    13
smart_242_raw    13
dtype: int64
削除前:  178298
削除後;  178285


# **学習を実装**

In [13]:
# データの準備
y_train = pd.DataFrame(df['failure']) #元がシリーズのためデータフレームに変換
y_test = pd.DataFrame(df_t['failure'])
X_train = df.drop(columns=['date','serial_number','failure']) #トレーニングデータ用に加工
X_test = df_t.drop(columns=['date','serial_number','failure'])

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178285 entries, 0 to 178297
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   smart_1_raw    178285 non-null  float64
 1   smart_4_raw    178285 non-null  float64
 2   smart_5_raw    178285 non-null  float64
 3   smart_7_raw    178285 non-null  float64
 4   smart_9_raw    178285 non-null  float64
 5   smart_12_raw   178285 non-null  float64
 6   smart_183_raw  178285 non-null  float64
 7   smart_184_raw  178285 non-null  float64
 8   smart_187_raw  178285 non-null  float64
 9   smart_188_raw  178285 non-null  float64
 10  smart_189_raw  178285 non-null  float64
 11  smart_190_raw  178285 non-null  float64
 12  smart_192_raw  178285 non-null  float64
 13  smart_193_raw  178285 non-null  float64
 14  smart_194_raw  178285 non-null  float64
 15  smart_197_raw  178285 non-null  float64
 16  smart_198_raw  178285 non-null  float64
 17  smart_199_raw  178285 non-nul

## **ランダムフォレストで分類を実装**

ハイパーパラメータの設定

In [14]:
n_estimators= 20,     # 用意する決定木モデルの数
max_features = "log2"  # ジニ係数を利用
max_depth =  10   # 決定木のノード深さの制限値

ランダムフォレストモデル構築

In [15]:
model = RandomForestClassifier(max_depth=10,
                               max_features="log2", 
                               n_estimators=20)
clf= model.fit(X_train,y_train)
clf_pred = clf.predict(X_test)
accuracy_score(y_test,clf_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.951403651456937

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def plot_confusion_matrix(predict, y_test):
  cm = confusion_matrix(y_test, predict)
  matrix = pd.DataFrame(cm)
  matrix.columns = [['予測_正常(0)', '予測_故障(1)']]
  matrix.index = [['実際_正常(0)', '実際_故障(1)']]
  return matrix

In [17]:
matrix = plot_confusion_matrix(clf_pred, y_test)
print(matrix)
print('Accuracy = ', accuracy_score(y_true=y_test, y_pred=clf_pred).round(decimals=3))
print('Precision = ', precision_score(y_true=y_test, y_pred=clf_pred, zero_division=0).round(decimals=3))
print('Recall = ', recall_score(y_true=y_test, y_pred=clf_pred).round(decimals=3))
print('F1 score = ', f1_score(y_true=y_test, y_pred=clf_pred).round(decimals=3))

         予測_正常(0) 予測_故障(1)
実際_正常(0)   166254     1554
実際_故障(1)     7110     3367
Accuracy =  0.951
Precision =  0.684
Recall =  0.321
F1 score =  0.437


# **考察①**


モデル自体のaccは非常に高く当てはまりが良いようにも思えるが、2×2のクロス集計で確認すると明らかに故障時の適合度が低いことがわかる。また感度特異度の関係から、感度は高いが、特異度が低く結果的に、十分に故障の予測が行えていないと推測できる。これらは講義中にも説明があったが、サンプリング不均衡による影響を受けていると言える。そこで下記コードからサンプリング不均衡に対する是正を実施する。アンダーダンプリングを用いてサンプリング不均衡を是正し、さらに学習時にグリッドサーチを用いることで、上記で示したモデルと比較してどのような変化が生じるかを確認する。(グリッドサーチは時間がかかりすぎるため断念しました。すみません。上記と同様のパラメータで比較いたしました)


## **アンダーサンプリング**

In [18]:
# アンダーサンプリング、トレーニングデータ
from imblearn.under_sampling import RandomUnderSampler

target = 'failure'
rs = RandomUnderSampler(random_state=10)
under_sampling ,_ = rs.fit_resample(df,df[target])

print('*'*20)
print('＜元のデータ＞')
print('正常の件数：%d'%len(df.query(f'{target}==0')))
print('故障の件数：%d'%len(df.query(f'{target}==1')))
print('*'*20)
print('＜アンダーサンプリング後のデータ＞')
print('正常の件数：%d'%len(under_sampling.query(f'{target}==0')))
print('故障の件数：%d'%len(under_sampling.query(f'{target}==1')));

********************
＜元のデータ＞
正常の件数：1013362
故障の件数：62707
********************
＜アンダーサンプリング後のデータ＞
正常の件数：62707
故障の件数：62707


In [19]:
# アンダーサンプリング、テストデータ
from imblearn.under_sampling import RandomUnderSampler

target = 'failure'
rs = RandomUnderSampler(random_state=10)
under_sampling_t ,_ = rs.fit_resample(df_t,df_t[target])

print('*'*20)
print('＜元のデータ＞')
print('正常の件数：%d'%len(df_t.query(f'{target}==0')))
print('故障の件数：%d'%len(df.query(f'{target}==1')))
print('*'*20)
print('＜アンダーサンプリング後のデータ＞')
print('正常の件数：%d'%len(under_sampling_t.query(f'{target}==0')))
print('故障の件数：%d'%len(under_sampling_t.query(f'{target}==1')));

********************
＜元のデータ＞
正常の件数：167808
故障の件数：62707
********************
＜アンダーサンプリング後のデータ＞
正常の件数：10477
故障の件数：10477


アンダーサンプリングのデータを元にランダムフォレスト実行

In [20]:
# データの準備
y_train = pd.DataFrame(under_sampling['failure']) #元がシリーズのためデータフレームに変換
y_test = pd.DataFrame(under_sampling_t['failure'])
X_train = under_sampling.drop(columns=['date','serial_number','failure']) #トレーニングデータ用に加工
X_test = under_sampling_t.drop(columns=['date','serial_number','failure'])

In [21]:
n_estimators= 20,     # 用意する決定木モデルの数
max_features = "log2"  # ランダムに指定する特徴量の数
max_depth =  10   # 決定木のノード深さの制限値

model = RandomForestClassifier(max_depth=10,
                               max_features="log2", 
                               n_estimators=20)
clf= model.fit(X_train,y_train)
clf_pred = clf.predict(X_test)
accuracy_score(y_test,clf_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.6647895389901689

In [22]:
matrix = plot_confusion_matrix(clf_pred, y_test)
print(matrix)
print('Accuracy = ', accuracy_score(y_true=y_test, y_pred=clf_pred).round(decimals=3))
print('Precision = ', precision_score(y_true=y_test, y_pred=clf_pred, zero_division=0).round(decimals=3))
print('Recall = ', recall_score(y_true=y_test, y_pred=clf_pred).round(decimals=3))
print('F1 score = ', f1_score(y_true=y_test, y_pred=clf_pred).round(decimals=3))

         予測_正常(0) 予測_故障(1)
実際_正常(0)     7973     2504
実際_故障(1)     4520     5957
Accuracy =  0.665
Precision =  0.704
Recall =  0.569
F1 score =  0.629


# **考察②**



アンダーサンプリングを用いれサンプル不均衡を是正した後に、再度ランダムフォレストを実施した。結果としてはaccは0.69まで低下した。全体の正解率はアンダーサンプリング前と比較して著名に低下していることがわかる。これはアンダーサンプリングにて正常サンプル数を削除し、故障サンプル数と同等にしたことでデータの中の故障サンプル割合が増えたことで相対的にaccは低下していることが推察される。しかしながら、故障に関する感度が向上していることも同時に伺える。これは故障に関しての分類精度自体は、アンダーサンプリング前のモデルと比較して向上していると示唆された。また。アンダーサンプリングには大きな問題点がある。元の多数派であるデータに対して削減を実施していることから、多数派データに対してバイアスが生じていると考えられる。アンダーサンプリング時のバイアス除去についてはAndrea Dal Pozzoloらが2015年のIEEE Symposiumでバイアスの除去方法を紹介してる。また講義中に実施したXGBoostと比較すると本解析で実施したランダムフォレストの方がモデル精度としては劣る結果が見られた。
そして、ここからの考察は完全な私見である。上記のような異常検知モデルや診断モデルにおいて、異常データが少ないことはしばしば見られる。この時サンプリング方法を工夫し、サンプル数を是正したとしても十分なaccを得ることは難しい場合が多い。こういった場面に遭遇した時、私はモデルを2つ作成すべきではないかと考える。感度が高いモデルと特異度の高いモデルである。いわゆるスクリーニング検査として感度が高いモデルを用いて分類を実施。さらに特異度の高いモデルを確定分類として用いるという方法である。この方法であれば、検査が2段階となる手間はあるが検査精度の向上は見込めるのではないかと考える。しかし学習データに用いるモデルは結果的に同様のデータを用いるため、根本的に都合良く感度が高いモデル、特異度が高いモデルを作成することが可能か否かもまた問題であると言える。